In [27]:
import time
import re
from IPython.display import clear_output
from PIL import Image, ImageDraw, ImageColor, ImageFont

In [3]:
WIDTH = 80
HEIGHT = 8

In [4]:
def new_grid(w=WIDTH, h=HEIGHT):
    return ['.' * w for r in range(1, h+1)]

In [5]:
def print_grid(grid, md=False, suppress_dots=False):
    if md:
        print('```')
    for row in grid:
        if suppress_dots:
            print(re.sub(r'\.', ' ', row))
        else:
            print(row)
    if md:
        print('```')

In [6]:
def top(grid, l, r):
    new_segment = ''
    for i in range(l-1, r):
        if grid[0][i] == '.':
            new_segment += '*'
        else:
            new_segment += '.'
    grid[0] = grid[0][:l-1] + new_segment + grid[0][r:]
    return grid

In [7]:
def left(grid, t, b):
    for i in range(t-1, b):
        if grid[i][0] == '.':
            grid[i] = '*' + grid[i][1:]
        else:
            grid[i] = '.' + grid[i][1:]
    return grid

In [8]:
def rotate_column(grid, c, raw_n):
    n = raw_n % len(grid)
    col = [row[c-1] for row in grid]
    new_col = col[-n:] + col[:-n]
    for i in range(len(grid)):
        grid[i] = grid[i][:c-1] + new_col[i] + grid[i][c:]
    return grid

In [9]:
def rotate_row(grid, r, raw_n):
    n = raw_n % len(grid[0])
    grid[r-1] = grid[r-1][-n:] + grid[r-1][:-n]
    return grid

In [10]:
command_dispatch = {'left': left, 'top': top,
                   'rotate row': rotate_row,
                   'rotate column': rotate_column}

In [11]:
def parse(command):
    cmd, a, b = command.rsplit(maxsplit=2)
    return cmd, int(a), int(b)  

In [63]:
def interpret(commands, grid=None, w=WIDTH, h=HEIGHT, 
              show_each_step=False, md=False, overprint=False):
    if grid is None:
        grid = new_grid(w, h)
    frames = []
    for cn, c in enumerate(commands):
        cmd, a, b = parse(c)
        if cmd in command_dispatch:
            command_dispatch[cmd](grid, a, b)
        else:
            raise ValueError('Unknown command')
        if show_each_step:
            frames += [draw_frame(grid, c, cn)]
    if show_each_step: 
        print('Final')
        print_grid(grid, md=md)
    frames[0].save('frames.gif', save_all=True, append_images=frames[1:], duration=200)
    return grid

In [44]:
cmds = [c.strip() for c in open('05-pixels.txt').readlines()]
g = interpret(cmds)
print_grid(g)

...****..............*...................*.....*..............*.................
......*..............*...................***..**..............*.................
......*.*****.*****.****.*****..****.....*.*.***.*****.*****.****.*****..****...
......*.....*.*...*..*.......*..*........*..**.*.....*.*...*..*.......*..*......
......*.*****.*...*..*...*****..*........*..*..*.*****.*...*..*...*****..*......
......*.*...*.*...*..*...*...*..*........*.....*.*...*.*...*..*...*...*..*......
...*..*.*..**.*...*..**..*..**..*........*.....*.*..**.*...*..**..*..**..*......
....**...**.*.*...*...**..**.*..*........*.....*..**.*.*...*...**..**.*..*......


In [60]:
im = Image.new('RGBA', (WIDTH * 10, HEIGHT * 10 + 21))

# make a blank image for the text, initialized to transparent text color
# txt = Image.new('RGBA', im.size, (255,255,255,0))
txt = Image.new('RGBA', im.size, (0, 0, 0, 0))

# get a font
fnt = ImageFont.truetype('Pillow/Tests/fonts/FreeMono.ttf', 18)
# get a drawing context
d = ImageDraw.Draw(txt)

# draw text, full opacity
d.text((1,1), "Command description", font=fnt, fill='black')

print(d.textsize("Command description", fnt))


draw = ImageDraw.Draw(im)
for (r, row) in enumerate(g):
    for (c, cell) in enumerate(row):
        rx = c * 10
        ry = r * 10 + 20
        if cell == '*':
            fill_colour = 'lime'
        else:
            fill_colour = 'black'
        draw.rectangle([rx, ry, rx + 10, ry + 10], fill=fill_colour, outline='orange')
            
# draw.line((0, 0) + im.size, fill=128)
# draw.line((0, im.size[1], im.size[0], 0), fill=128)

out = Image.alpha_composite(im, txt)
out.save('test.png', 'PNG')

del draw



(209, 17)


In [76]:
def draw_frame(grid, command, frame_number, gif=True):
    im = Image.new('RGBA', (WIDTH * 10, HEIGHT * 10 + 21))

    # make a blank image for the text, initialized to transparent text color
    if gif:
        txt = Image.new('RGBA', im.size, (255,255,255,0))
    else:
        txt = Image.new('RGBA', im.size, (0, 0, 0, 0)) # use this line rather than line above for animated png

    # get a font
    fnt = ImageFont.truetype('Pillow/Tests/fonts/FreeMono.ttf', 18)
    # get a drawing context
    d = ImageDraw.Draw(txt)

    # draw text, full opacity
    if gif:
        d.text((1,1), command, font=fnt, fill='white')
    else:
        d.text((1,1), command, font=fnt, fill='black') # use this line rather than line above for animated png

    draw = ImageDraw.Draw(im)
    for (r, row) in enumerate(grid):
        for (c, cell) in enumerate(row):
            rx = c * 10
            ry = r * 10 + 20
            if cell == '*':
                fill_colour = 'lime'
            else:
                fill_colour = 'black'
            draw.rectangle([rx, ry, rx + 10, ry + 10], fill=fill_colour, outline=128)

    # draw.line((0, 0) + im.size, fill=128)
    # draw.line((0, im.size[1], im.size[0], 0), fill=128)

    out = Image.alpha_composite(im, txt)
    if not gif:
        out.save('frame{:04}.png'.format(frame_number), 'PNG') # uncomment here to save animated png frames

    del draw
    
    return out

In [24]:
g

['...****..............*...................*.....*..............*.................',
 '......*..............*...................***..**..............*.................',
 '......*.*****.*****.****.*****..****.....*.*.***.*****.*****.****.*****..****...',
 '......*.....*.*...*..*.......*..*........*..**.*.....*.*...*..*.......*..*......',
 '......*.*****.*...*..*...*****..*........*..*..*.*****.*...*..*...*****..*......',
 '......*.*...*.*...*..*...*...*..*........*.....*.*...*.*...*..*...*...*..*......',
 '...*..*.*..**.*...*..**..*..**..*........*.....*.*..**.*...*..**..*..**..*......',
 '....**...**.*.*...*...**..**.*..*........*.....*..**.*.*...*...**..**.*..*......']

In [77]:
cmds = [c.strip() for c in open('05-pixels.txt').readlines()]
g = interpret(cmds, show_each_step=True)
print_grid(g)

Final
...****..............*...................*.....*..............*.................
......*..............*...................***..**..............*.................
......*.*****.*****.****.*****..****.....*.*.***.*****.*****.****.*****..****...
......*.....*.*...*..*.......*..*........*..**.*.....*.*...*..*.......*..*......
......*.*****.*...*..*...*****..*........*..*..*.*****.*...*..*...*****..*......
......*.*...*.*...*..*...*...*..*........*.....*.*...*.*...*..*...*...*..*......
...*..*.*..**.*...*..**..*..**..*........*.....*.*..**.*...*..**..*..**..*......
....**...**.*.*...*...**..**.*..*........*.....*..**.*.*...*...**..**.*..*......
...****..............*...................*.....*..............*.................
......*..............*...................***..**..............*.................
......*.*****.*****.****.*****..****.....*.*.***.*****.*****.****.*****..****...
......*.....*.*...*..*.......*..*........*..**.*.....*.*...*..*.......*..*......
......*.*****.*...*..*

Create animated png.

In [ ]:
! apngasm frames.png frame0*png 1 5